In [1]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")
import json
import pandas as pd
import requests
import logging
logging.getLogger('requests').setLevel(logging.CRITICAL)

In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = [
    'http://localhost:9201'
])

In [4]:
es.info()

ObjectApiResponse({'name': 'Pranshus-MacBook-Air.local', 'cluster_name': 'elasticsearch_joker', 'cluster_uuid': '4Db_3SArQvavfi4B4Hb91g', 'version': {'number': '7.17.3', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff', 'build_date': '2022-04-19T08:11:19.070913226Z', 'build_snapshot': False, 'lucene_version': '8.11.1', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'})

In [5]:
import numpy as np
import pandas as pd

df = pd.read_pickle("final1.pkl")

In [6]:
df

,Id,CreationDate,Score,Title,Body,Tag,title_vec
0,80,2008-08-01T13:57:07Z,26,SQLStatement.execute() - multiple queries in o...,Ive written database generation script SQL wan...,"flex,actionscript-3,air","[-0.5923703908920288, 0.18966519832611084, 0.0..."
1,90,2008-08-01T14:41:24Z,144,Good branching and merging tutorials for Torto...,Are really good tutorials explaining branching...,"svn,tortoisesvn,branch,branching-and-merging","[-0.18364280462265015, 0.031254399567842484, 0..."
2,120,2008-08-01T15:50:08Z,21,ASP.NET Site Maps,Has anyone got experience creating SQLbased AS...,"sql,asp.net,sitemap","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,180,2008-08-01T18:42:19Z,53,Function for creating color wheels,This something Ive pseudosolved many times nev...,"algorithm,language-agnostic,colors,color-space","[-0.10257666558027267, 0.21121667325496674, 0...."
4,260,2008-08-01T23:22:08Z,49,Adding scripting functionality to .NET applica...,I little game written C It uses database backe...,"c#,.net,scripting,compiler-construction","[-0.5590633749961853, 0.12501268088817596, -0...."
...,...,...,...,...,...,...,...
1102563,40142860,2016-10-19T23:01:07Z,0,Adding large data in Excel,I around 200000 data excel separated per 15min...,"excel,excel-formula","[-0.4378100335597992, 0.1316293329000473, 0.09..."
1102564,40142900,2016-10-19T23:05:07Z,0,LinkedIN Encounter error: Your application has...,I able connect LinkedIn API two months everyth...,"api,oauth,linkedin","[-0.1463644653558731, 0.04537837207317352, -0...."
1102565,40142910,2016-10-19T23:05:49Z,0,Validation for must_be_below_user_limit allowi...,So I Am building multitenant app Rails 4 Apart...,"ruby-on-rails,validation,ruby-on-rails-4","[-0.4635421633720398, 0.23969916999340057, -0...."
1102566,40142940,2016-10-19T23:08:42Z,-1,Drawing an iscosceles triangle of asteriks on C++,I learning c Im trying draw iscosceles triangl...,c++,"[-0.039291754364967346, 0.08876225352287292, 0..."


In [21]:
try:
    es.indices.create(index = 'so-vec-ret')
except:
    pass

In [10]:
df.columns

Index(['Id', 'CreationDate', 'Score', 'Title', 'Body', 'Tag', 'title_vec'], dtype='object')

In [11]:
df.dtypes

Id               int64
CreationDate    object
Score            int64
Title           object
Body            object
Tag             object
title_vec       object
dtype: object

In [16]:
from json import JSONEncoder


In [19]:
class NumpyArrayEncoder(JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return JSONEncoder.default(self, obj)

In [14]:
df_split = np.array_split(df, 4)

In [15]:
index = 'so-vec-ret'
typ = 'test'
server = 'http://localhost:9201'

headers = {'content-type': 'application/x-ndjson', 'Accept-Charset': 'UTF-8'}
records = df_split[0].to_dict(orient='records')
actions = ["""{ "index" : { "_index" : "%s", "_type" : "%s"} }\n""" % (index, typ) +json.dumps(records[j])
                for j in range(len(records))]

In [28]:
chunk_size = 5000

#actions1 = [json.dumps(records[j], cls=NumpyArrayEncoder) for j in ]
        
i=0
while i<len(actions):
    if(i%10000 == 0):
        print(i)
    try:
        serverAPI = server + '/_bulk' 
        data='\n'.join(actions[i:min([i+chunk_size,len(actions)])])
        data = data + '\n'
        r = requests.post(serverAPI, data = data, headers=headers)
        #print (r.content)
    except:
        break
    i = i+chunk_size

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000


In [ ]:
index = 'so-vec-ret'
typ = 'test'
server = 'http://localhost:9201'

headers = {'content-type': 'application/x-ndjson', 'Accept-Charset': 'UTF-8'}
records = df_split[1].to_dict(orient='records')
actions = ["""{ "index" : { "_index" : "%s", "_type" : "%s"} }\n""" % (index, typ) +json.dumps(records[j])
                for j in range(len(records))]

chunk_size = 5000

#actions1 = [json.dumps(records[j], cls=NumpyArrayEncoder) for j in ]
        
i=0
while i<len(actions):
    if(i%10000 == 0):
        print(i)
    try:
        serverAPI = server + '/_bulk' 
        data='\n'.join(actions[i:min([i+chunk_size,len(actions)])])
        data = data + '\n'
        r = requests.post(serverAPI, data = data, headers=headers)
        #print (r.content)
    except:
        break
    i = i+chunk_size

In [ ]:
index = 'so-vec-ret'
typ = 'test'
server = 'http://localhost:9201'

headers = {'content-type': 'application/x-ndjson', 'Accept-Charset': 'UTF-8'}
records = df_split[2].to_dict(orient='records')
actions = ["""{ "index" : { "_index" : "%s", "_type" : "%s"} }\n""" % (index, typ) +json.dumps(records[j])
                for j in range(len(records))]
chunk_size = 5000

#actions1 = [json.dumps(records[j], cls=NumpyArrayEncoder) for j in ]
        
i=0
while i<len(actions):
    if(i%10000 == 0):
        print(i)
    try:
        serverAPI = server + '/_bulk' 
        data='\n'.join(actions[i:min([i+chunk_size,len(actions)])])
        data = data + '\n'
        r = requests.post(serverAPI, data = data, headers=headers)
        #print (r.content)
    except:
        break
    i = i+chunk_size

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000


In [ ]:
index = 'so-vec-ret'
typ = 'test'
server = 'http://localhost:9201'

headers = {'content-type': 'application/x-ndjson', 'Accept-Charset': 'UTF-8'}
records = df_split[3].to_dict(orient='records')
actions = ["""{ "index" : { "_index" : "%s", "_type" : "%s"} }\n""" % (index, typ) +json.dumps(records[j])
                for j in range(len(records))]

chunk_size = 5000

#actions1 = [json.dumps(records[j], cls=NumpyArrayEncoder) for j in ]
        
i=0
while i<len(actions):
    if(i%10000 == 0):
        print(i)
    try:
        serverAPI = server + '/_bulk' 
        data='\n'.join(actions[i:min([i+chunk_size,len(actions)])])
        data = data + '\n'
        r = requests.post(serverAPI, data = data, headers=headers)
        #print (r.content)
    except:
        break
    i = i+chunk_size

In [ ]:
try:
    b = {
        "mappings": {
            "properties": {
                "Title": {
                    "type": "text"
                },
                "title_vec": {
                    "type": "dense_vector",
                    "dims": 300
                }
            }
        }
    }


    ret = es.indices.create(index='so-vec-ret', ignore=400, body=b) #400 caused by IndexAlreadyExistsException, 
    print(json.dumps(ret,indent=4))
except:
    print("cant")